# Naive Bayes Classifier

1. **Réaliser un TF-IDF** sur l'ensemble des `cases_texte` du `df_final`.
2. **Tenter de prédire** la colonne `major_mesh_terms` à partir du TF-IDF.

## Import data

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # Barre de progression pour Jupyter
pd.set_option('display.max_colwidth', None)
import s3fs


In [2]:
AWS = False

In [3]:
if os.path.exists("../data/df_train.pkl"):
    data_train = pd.read_pickle('../data/df_train.pkl')
    print(data_train.shape)

if os.path.exists("../data/df_test.pkl"):
    data_test = pd.read_pickle('../data/df_test.pkl')
    print(data_test.shape)

(9646, 12)
(1072, 12)


In [4]:
os.environ["AWS_ACCESS_KEY_ID"] = 'TX7EAFIX4BX0YG40M0HS'
os.environ["AWS_SECRET_ACCESS_KEY"] = '6u6aMhddxRy+JkoE1+Gb53Rf+PqPync9xgyO6q3W'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJUWDdFQUZJWDRCWDBZRzQwTTBIUyIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzQyNTczNTU4LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InF1ZW50aW4ubWFycmV0QGVuc2FlLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTc0MzYxMDc3OCwiZmFtaWx5X25hbWUiOiJNYXJyZXQiLCJnaXZlbl9uYW1lIjoiUXVlbnRpbiIsImdyb3VwcyI6WyJVU0VSX09OWVhJQSJdLCJpYXQiOjE3NDMwMDU5NzgsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiJjYWQ4MDk4OS05ZTY4LTQ1MzAtODlhYS0wM2UxZTliZTg3ZGQiLCJsb2NhbGUiOiJmciIsIm5hbWUiOiJRdWVudGluIE1hcnJldCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJxdWVudGluMTk5OSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2lkIjoiZTlmMWI0YzgtNTYwNC00M2UwLTk2NDUtZjM0M2Q2Y2U0NDVmIiwic3ViIjoiOGM1NjkxMjYtMGI2Mi00MGI0LTg4ZDAtMWFhYzgyMTJiOGE5IiwidHlwIjoiQmVhcmVyIn0.Gl-pro2vlaMpQZiX9x1b9ngp10WD8yH8_KIi8VqofdmnaJ67cdOyIEQ_2NJ-V-XZAjSCC6fC8L7sD8POngUs6g'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

In [5]:
print(repr(os.getenv("AWS_ACCESS_KEY_ID")))
print(repr(os.getenv("AWS_SECRET_ACCESS_KEY")))

'TX7EAFIX4BX0YG40M0HS'
'6u6aMhddxRy+JkoE1+Gb53Rf+PqPync9xgyO6q3W'


raise RuntimeError("⛔ Cette cellule ne doit pas être exécutée.")

#Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET_OUT = "s3://quentin1999/Data_Projet_NLP"
FILE_KEY_OUT_S3 = "df_final"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df.to_csv(file_out)

In [6]:
if AWS == True:
    fs.ls("s3://quentin1999/Data_Projet_NLP")

In [7]:
if AWS == True:
    if not os.path.exists("../data/df_target_V2.pkl"):
        BUCKET = "s3://quentin1999/Data_Projet_NLP"
        FILE_KEY_S3 = "df_target_V2.pkl"
        FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

        with fs.open(FILE_PATH_S3, mode="rb") as file_in:
            df = pd.read_pickle(file_in)

        print("✅ Fichier chargé avec succès, shape :", df.shape)

        df.to_pickle("data/df_target_V2.pkl")

        print("✅ Fichier sauvegardé avec succès dans le dossier data")

## Naive Bayes Classifier

### Bag-of-word representation

In [8]:
from sklearn.model_selection import train_test_split

print ('train size',data_train.shape)
print ('test size', data_test.shape)

# create features X and target y
X_train = data_train.case_text
X_test = data_test.case_text

y_train = data_train.target
y_test = data_test.target

train size (9646, 12)
test size (1072, 12)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(max_features=1000)
#vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=3000)
# fit on train data
vectorizer.fit(X_train)

# Apply it on train and dev data
X_train_counts = vectorizer.transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [12]:
vectorizer

CountVectorizer(max_features=1000)

### OneVsRestClassifier avec CountVectorizer

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Initiate the model
# Définition du modèle multi-label simple avec Logistic Regression
model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))

# Train
model.fit(X_train_counts, np.vstack(y_train.values))
y_pred = (model.predict_proba(X_test_counts) > 0.5).astype(int)

/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 1 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 2 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 20 is present in all training examples.
  warnings.warn(


In [15]:
from sklearn.metrics import f1_score, hamming_loss, classification_report
import numpy as np

# 1. Conversion correcte de y_test (stocké sous forme de vecteurs dans une colonne)
y_test_array = np.vstack(y_test.values)

# 2. Calcul des métriques
f1_micro = f1_score(y_test_array, y_pred, average='micro')
f1_macro = f1_score(y_test_array, y_pred, average='macro')
hamming = hamming_loss(y_test_array, y_pred)

# 3. Affichage des métriques globales
print(f"F1 micro       : {f1_micro:.4f}")
print(f"F1 macro       : {f1_macro:.4f}")
print(f"Hamming loss   : {hamming:.4f}")

# 4. Classification report
print("\nClassification Report :")
print(classification_report(y_test_array, y_pred, zero_division=0))

F1 micro       : 0.5582
F1 macro       : 0.3790
Hamming loss   : 0.0807

Classification Report :
              precision    recall  f1-score   support

           0       0.72      0.62      0.67       233
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.78      0.75      0.76       333
           4       0.42      0.36      0.39        89
           5       0.62      0.54      0.58       159
           6       0.37      0.36      0.37        47
           7       0.64      0.55      0.59       155
           8       0.29      0.31      0.30        32
           9       0.73      0.57      0.64       213
          10       0.76      0.49      0.60        79
          11       0.57      0.56      0.56       129
          12       0.00      0.00      0.00         0
          13       0.77      0.61      0.68       213
          14       0.58      0.46      0.51       136
          15       0.47      0.31     

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### OneVsRestClassifier avec TF-IDF ngram = 2

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
# fit on train data
vectorizer.fit(X_train)

# Apply it on train and dev data
X_train_counts = vectorizer.transform(X_train)
X_test_counts = vectorizer.transform(X_test)

In [19]:
# Initiate the model
# Définition du modèle multi-label simple avec Logistic Regression
model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))

# Train
model.fit(X_train_counts, np.vstack(y_train.values))
y_pred = (model.predict_proba(X_test_counts) > 0.25).astype(int)

# 1. Conversion correcte de y_test (stocké sous forme de vecteurs dans une colonne)
y_test_array = np.vstack(y_test.values)

# 2. Calcul des métriques
f1_micro = f1_score(y_test_array, y_pred, average='micro')
f1_macro = f1_score(y_test_array, y_pred, average='macro')
hamming = hamming_loss(y_test_array, y_pred)

# 3. Affichage des métriques globales
print(f"F1 micro       : {f1_micro:.4f}")
print(f"F1 macro       : {f1_macro:.4f}")
print(f"Hamming loss   : {hamming:.4f}")

# 4. Classification report
print("\nClassification Report :")
print(classification_report(y_test_array, y_pred, zero_division=0))

/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 1 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 2 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 20 is present in all training examples.
  warnings.warn(


F1 micro       : 0.6745
F1 macro       : 0.4535
Hamming loss   : 0.0657

Classification Report :
              precision    recall  f1-score   support

           0       0.75      0.86      0.80       233
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.79      0.89      0.84       333
           4       0.57      0.39      0.47        89
           5       0.75      0.79      0.77       159
           6       0.73      0.23      0.35        47
           7       0.76      0.70      0.73       155
           8       0.78      0.22      0.34        32
           9       0.71      0.76      0.73       213
          10       0.87      0.58      0.70        79
          11       0.66      0.70      0.68       129
          12       0.00      0.00      0.00         0
          13       0.76      0.72      0.74       213
          14       0.72      0.60      0.65       136
          15       0.55      0.48     

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
